# TAMU Datathon: Taco/Burrito Challenge

### Team Name: Taco 'Bout It!
### Team Members: Alex Riley, Jacqueline Antwi-Danso

We decided to participate in the Goldman Sachs data challenge, which centers on a dataset logging taco and burrito menu items in the United States ([Kaggle link](https://www.kaggle.com/datafiniti/restaurants-burritos-and-tacos/)). The tasks of the challenge are:

```
The final product of your efforts should include a visualization of your output, with supporting documentation detailing the modeling and analysis performed.
```

We'll start with the usual Python imports, plus some that will be useful for data cleaning (dealing with zip codes).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from uszipcode import SearchEngine
import plotly.graph_objects as go
%matplotlib inline

# so plotly map can render
import plotly
plotly.offline.init_notebook_mode(connected=True)

# contains mappings between state name and abbreviation
import utils

# Take a look at data

In [ ]:
file = 'data/just-tacos-and-burritos.csv'
data = pd.read_csv(file)

In [ ]:
data.head(5)

There's a lot of unnamed columns that are filled with `NaN`. Let's get rid of those.

In [ ]:
empty = data.isna().sum() == len(data)
assert np.array(["Unnamed" in col for col in data.columns[empty]]).all()
data.drop(columns=data.columns[empty], inplace=True)

### List of columns

* `id`: unique ID for restaurant
* `address`: restaurant address (number and street name)
* `categories`: categories for restaurant (e.g. "Restaurant" or "Restaurant Delivery")
* `city`: city name
* `country`: country (note: all are in the US)
* `cuisines`: type of restaurant, e.g. "Coffee" or "Mexican". Not unique (one example is "Buffets, Pizza")
* `dateAdded`: date that entry was added to dataset
* `dateUpdated`: date that entry was last updated (can be equal to `dateAdded`)
* `keys`: ???
* `latitude`: latitude of the restaurant
* `longitude`: longitude of the restaurant
* `menuPageURL`: URL to menu
* `menus.amountMax`: max amount on menu? (sparsely filled; 37,000 NaN)
* `menus.amountMin`: min amount on menu? (sparsely filled; 37,000 NaN)
* `menus.category`: category that item falls under in menu (e.g. "Main Course", "Tacos"). Sparsely filled, 73,531 NaN
* `menus.currency`: currency used on item. usually USD, 16 entries are EUR
* `menus.dateseen`: date that menu was observed
* `menus.description`: description of item in menu
* `name`: name of restaurant
* `postalCode`: ZIP code of restaurant
* `priceRangeCurrency`: currency used for `menus.priceRangeMin/Max` usually USD, one entry in AUD
* `priceRangeMin`: minimum price of items on menu
* `priceRangeMax`: maximum price of items on menu
* `province`: typically state but not always. needs cleaning
* `websites`: website for the restaurant

### Potential data cleaning issues
* `name` can have multiple values, like `McDonald's` and `Mc Donalds`
* many columns are incomplete, including `postalCode` and `latitude/longitude` which might make analysis/visualizing the spatial distribution of restaurants difficult

## Cleaning the data

Consistent identification of city + state (`province` is not clean version of this). We'll start off by creating a new column named `state`.

**Note: this section can be skipped if it's already been run once**

In [ ]:
data['state'] = data['province']

# three entries had no province info, all were in San Francisco
data.loc[data['state'].isna(), 'state'] = 'CA'

Now we have a few freebies. These were common (top 25-ish) values for `province` that are easily mapped to states, as well as `province` values that were 2 characters that did not match state abbreviation codes.

In [ ]:
data.loc[data['state'] == 'California', 'state'] = 'CA'
data.loc[data['state'] == 'Manhattan', 'state'] = 'NY'
data.loc[data['state'] == 'New York City', 'state'] = 'NY'
data.loc[data['state'] == 'Ny', 'state'] = 'NY'
data.loc[data['state'] == 'Ls', 'state'] = 'MO'

The pops that remain would be a pain to continue for. For these ~8000 pops, we will use the `uszipcode` package to map provided zip codes to the state.

In [ ]:
badmask = data['state'].apply(len) != 2
search = SearchEngine()
data.loc[badmask, 'state'] = data[badmask].apply(lambda x: search.by_zipcode(x['postalCode']).state if x['postalCode'] else x['state'], axis=1)

And we're done! We can check that all of the `state` items are valid state codes by cross-referencing against the list located in `utils.py`.

In [ ]:
data['state'].apply(lambda x: True if x in utils.abbrev_us_state else False).all()

In [ ]:
data['citystate'] = data.apply(lambda x: x['city']+', '+x['state'], axis=1)

# Question: Where are the authentic Mexican restaurants?

## Marking out "authentic"

We want to exclude stores that can be reliably marked as "inauthentic," like Subway or McDonald's. For this, we'll exclude any restaurants from this list of the [32 biggest fast food chains in America](https://www.qsrmagazine.com/content/32-biggest-fast-food-chains-america). We also opt to include Chili's in the list of excluded chains.

Notice, some names have permutations that match names occurring in top 100 (e.g. McDonald's).

In [ ]:
def chains_mask(data):
    exclude_list = ["Subway", "Starbucks",
                "McDonald's", "Mcdonald's", "Mc Donald's", "Mcdonalds", "McDonalds",
                "Dunkin", "Pizza Hut", "Burger King", "Wendy's", "Taco Bell",
                "Domino's", "Dairy Queen", "Little Caesars", "KFC",
                "Sonic Drive In", "SONIC Drive In", "Sonic Drive-in", "Sonic Drive-In",
                "Papa John's", "Arby's", "Jimmy John's",
                "Baskin-Robbins", "Chipotle Mexican Grill", "Chick-Fil-A", "Popeye's",
                "Jack in the Box", "Jack In The Box",
                "Panda Express", "Panera", "Carl's Jr.", "Jersey Mike's", "Papa Murphy's",
                "Five Guys", "Auntie Anne's", "Wingstop", "Firehouse Subs"]

    # also exclude Chili's
    exclude_list.append("Chili's Grill & Bar")
    exclude_list.append("Chili's Grill Bar")
    exclude_list.append("Chili's")
    exclude_list.append("Chili's Too")

    chain = [False] * len(data)
    for name in exclude_list:
        chain |= data['name'] == name
    
    return ~chain

authentic = data[chains_mask(data)]

Now we are interested in the question where are authentic **restaurants** concentrated in the U.S.? For this, we need to only have a list of authentic restaurants, not a list of authentic burritos/tacos. Luckily, we can just mask duplicated `id`s.

In [ ]:
unique_restaurant_mask = ~authentic['id'].duplicated()
restaurants = authentic[unique_restaurant_mask]

Now we can get a very simple answer for which cities host the most authentic Mexican restaurants in the U.S.:

In [ ]:
citycounts = restaurants['citystate'].value_counts()
citycounts.head(7)

However, this just looks like a list of big cities with a lot of people (who would therefore have a lot of authentic Mexican restaurants). To fix this, we can instead try to get the number of restaurants _per capita_.

For population data, we'll use [population estimates from the U.S. Census Bureau](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-cities-and-towns.html#ds) for 2018. 

In [ ]:
popfile = 'data/sub-est2018_all.csv'
popdata = pd.read_csv(popfile, encoding="ISO-8859-1")
popdata.head()

So we simply need to find a city by matching the city name and state in the `popdata` table. We also need to convert the state code (e.g. "AL") to a state name ("Alabama").

In [ ]:
population = []
for name, pop in citycounts.iteritems():
    match = popdata['NAME'] == name[:-4] + ' city'
    match |= popdata['NAME'] == name[:-4] + ' town'
    match |= popdata['NAME'] == name[:-4] + ' village'
    match &= popdata['STNAME'] == utils.abbrev_us_state[name[-2:]]
    pop = np.max(popdata[match]['POPESTIMATE2018'])
    population.append(pop)
population = np.array(population)

In [ ]:
citycounts_percapita = (citycounts/population) * 1000
citycounts_percapita.sort_values(ascending=False).head(7)

So these are the places that have a lot of authentic Mexican restaurants relative to how big their population is. However, all of these places are very small and isolated. Let's place a cut on the data requiring a population of over 50,000 (the high end of what is considered the threshold for a "city")

In [ ]:
threshold = 50000
citycounts_percapita[population > threshold].sort_values(ascending=False).head(7)

As can be (somewhat) expected, the list is now dominated by cities in California (with one entry from New Mexico).

## Visualization: where are the tacos?

In [ ]:
cities = data['city'].unique().tolist()

In [ ]:
def food_nums(loc_arr):
    df = data
    tacos = []; burritos = []
    for name in loc_arr:
        menu_options = df[df['city'] == name]['menus.name']
        
        # for each restaurant in each city, calculate the number of burritos and tacos 
        num_tacos = []
        num_burritos = []
        for option in menu_options: 
             
            if "Taco" in option:
                num_tacos.append(1)
            
            if "Burrito" in option:
                num_burritos.append(1)
            
        if len(num_tacos) != 0:
            total_tacos = np.sum(num_tacos)
        else:
            total_tacos = 0
        if len(num_burritos) != 0:
            total_burritos = np.sum(num_burritos)
        else: 
            total_burritos = 0
           
        tacos.append(total_tacos)
        burritos.append(total_burritos)
        
    return tacos, burritos

In [ ]:
num_tacos, num_burritos = food_nums(cities)

In [ ]:
# turn zeros into very big/small numbers to avoid to avoid division by 0
num_tacos = np.array(num_tacos)
num_burritos = np.array(num_burritos)

tmp_num_tacos = np.copy(num_tacos)
tmp_num_burritos = np.copy(num_burritos)

tmp_num_tacos[tmp_num_tacos == 0] = -1
tmp_num_burritos[tmp_num_burritos == 0] = -1

In [ ]:
city_ratio = tmp_num_burritos/tmp_num_tacos

In [ ]:
city_ratio[city_ratio < 0] = np.inf

In [ ]:
# find unique lon and lat for each city 
lon = []; lat = []
for city in cities:
        lon.append(np.unique(data[data['city'] == city]['longitude'])[0])
        lat.append(np.unique(data[data['city'] == city]['latitude'])[0])

In [ ]:
# renaming because plotly has similar keyword 
lon_arr = np.copy(lon); lat_arr = np.copy(lat)

In [ ]:
tmp_cities = np.array(cities)[~np.isinf(city_ratio)]

In [ ]:
text_arr = [city + '\n B/T:' + str(np.round_(num, decimals = 2)) for city, num in zip(cities, city_ratio)]

In [ ]:
fig = go.Figure()
limits = [(0,3),(4,7),(8,11),(12,15),(16,20)]
colors = ["brown","magenta","cyan","orange","green"]

for i in range(len(limits)):
    lim = limits[i]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = lon_arr,
        lat = lat_arr,
        text = text_arr,
        marker = dict(
            size = city_ratio*5,
            color = colors[i],
            sizemode = 'area', 
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))
    
fig.update_layout(
        #title_text = 'Menu options by city <br>(Click legend to populate map)',
        title_text = 'Menu options by city <br>(Hover on point to see burrito/taco ratio)',
        showlegend = False,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',))